In [17]:
from db_utils import retrieve_data
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import sys
sys.path.insert(0, '../')
from features import build_features
from model.train import train_and_pkl
path = r"c:\Users\brtnl\OneDrive\Desktop\code\sports_betting"
os.chdir(path)

pd.set_option('display.max_columns', None)

In [ ]:
# The final will be merged venue and game df.
venue_df = retrieve_data("cfb", "venues")
game_df = retrieve_data("cfb", "games")
df = pd.merge(game_df, venue_df, "left", left_on=["venue_id", "venue"], right_on=["id", "name"], suffixes=("", "_venue"))

In [ ]:
# df.columns# Drop rows where either 'home_line_scores' or 'away_line_scores' is an empty list
df = df[df['home_line_scores'].apply(lambda x: x != []) & df['away_line_scores'].apply(lambda x: x != [])]

# Display the updated DataFrame
print(df)


In [60]:
# df.head()

In [ ]:
# cleaning_steps
df = df.dropna(subset=['home_points', 'away_points'])
df = df[df['home_line_scores'].apply(lambda x: x != []) & df['away_line_scores'].apply(lambda x: x != [])]

In [72]:
df['ot'] = df['home_line_scores'].apply(lambda x: len(x) > 4)
df['home_line_scores'] = df['home_line_scores'].apply(
    lambda x: x if len(x) == 5 else x + [0]
)

In [73]:
# # Find rows where 'home_line_scores' is not a list of length 4
# invalid_rows = df[~df['home_line_scores'].apply(lambda x: isinstance(x, list) and len(x) == 4)]


In [77]:
min(df['home_line_scores'].apply(lambda x: len(x)))

1

In [79]:
# Find rows where 'home_line_scores' is an empty list
empty_lists = df[df['home_line_scores'].apply(lambda x: x == [0])]


In [80]:
empty_lists

,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_classification,home_points,home_line_scores,home_postgame_win_probability,home_pregame_elo,home_postgame_elo,away_id,away_team,away_conference,away_classification,away_points,away_line_scores,away_postgame_win_probability,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes,id_venue,name,city,state,zip,countrycode,latitude,longitude,capacity,dome,timezone,elevation,constructionyear,grass,ot
7946,401029912,2018,2,regular,2018-09-08,False,True,False,False,NaN,3991.0,"William Dick"""" Price Stadium""""",2450,Norfolk State,MEAC,fcs,0.0,[0],NaN,NaN,NaN,256,James Madison,CAA,fcs,17.0,[],NaN,NaN,NaN,NaN,,None,3991.0,"William Dick"""" Price Stadium""""",Norfolk,VA,23504,US,36.846944,-76.260084,30000.0,False,America/New_York,4.916433,1997.0,False,False
8106,401030482,2018,3,regular,2018-09-15,False,True,False,False,NaN,3599.0,Ace W. Mumford Stadium,2582,Southern,SWAC,fcs,33.0,[0],NaN,NaN,NaN,2324,Langston University,None,None,18.0,[],NaN,NaN,NaN,NaN,,None,3599.0,Ace W. Mumford Stadium,Baton Rouge,LA,70807,US,30.522146,-91.189609,28500.0,False,America/Chicago,20.242132,1928.0,False,False
9867,401128943,2019,5,regular,2019-09-28,False,True,False,False,NaN,4899.0,Fifth Third Stadium,338,Kennesaw State,Big South,fcs,31.0,[0],NaN,NaN,NaN,2890,Reinhardt University,None,None,7.0,[],NaN,NaN,NaN,NaN,,CALLED IN 3RD QUARTER (SEVERE WEATHER),4899.0,Fifth Third Stadium,Kennesaw,GA,48187,US,34.028932,-84.567623,8318.0,False,None,296.958984,2010.0,None,False
12099,401329015,2021,1,regular,2021-09-02,False,True,False,False,NaN,5935.0,Parker Field at McCool Stadium,2170,Delta State,Gulf South,ii,48.0,[0],NaN,NaN,NaN,2064,Bethel University Tennessee,None,None,14.0,[],NaN,NaN,NaN,NaN,,None,5935.0,Parker Field at McCool Stadium,Cleveland,MS,38732,US,33.744002,-90.724819,8125.0,False,None,NaN,NaN,None,False
12221,401329092,2021,1,regular,2021-09-04,False,True,False,False,NaN,3612.0,Alumni Memorial Stadium (NC),2940,Livingstone,CIAA,ii,10.0,[0],NaN,NaN,NaN,2805,Clark Atlanta,SIAC,ii,13.0,[],NaN,NaN,NaN,NaN,,None,3612.0,Alumni Memorial Stadium (NC),Salisbury,NC,28144,US,35.670973,-80.474226,6000.0,False,None,NaN,NaN,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23867,401676317,2024,7,regular,2024-10-12,False,True,False,True,NaN,6153.0,Hanson Stadium,205,Pacific (OR),Northwest,iii,18.0,[0],NaN,NaN,NaN,2486,Pacific Lutheran,Northwest,iii,34.0,[],NaN,NaN,NaN,NaN,,None,6153.0,Hanson Stadium,Forest Grove,OR,None,US,45.519745,-123.111441,1100.0,False,None,NaN,NaN,None,False
24900,401675593,2024,11,regular,2024-11-09,False,True,False,True,NaN,6056.0,B.T. Harvey Stadium,60,Morehouse College,SIAC,ii,17.0,[0],NaN,NaN,NaN,2805,Clark Atlanta,SIAC,ii,28.0,[],NaN,NaN,NaN,NaN,,None,6056.0,B.T. Harvey Stadium,Atlanta,GA,30315,US,33.748992,-84.390264,9000.0,False,None,NaN,NaN,None,False
24992,401675803,2024,11,regular,2024-11-09,False,True,False,True,NaN,3612.0,Alumni Memorial Stadium (NC),2940,Livingstone,CIAA,ii,15.0,[0],NaN,NaN,NaN,2304,Johnson C Smith,CIAA,ii,10.0,[],NaN,NaN,NaN,NaN,,None,3612.0,Alumni Memorial Stadium (NC),Salisbury,NC,28144,US,35.670973,-80.474226,6000.0,False,None,NaN,NaN,None,False
25311,401675953,2024,12,regular,2024-11-16,False,True,False,True,NaN,5900.0,Don and Nona Williams Stadium,2744,Wisconsin-Stout,Wisconsin,iii,38.0,[0],NaN,NaN,NaN,272,Wisconsin-Platteville,Wisconsin,iii,46.0,[],NaN,NaN,NaN,NaN,,None,5900.0,Don and Nona Williams Stadium,Menomonie,WI,54751,US,44.875518,-91.919342,4500.0,False,None,NaN,NaN,None,False


In [74]:
df[['home_q1', 'home_q2', 'home_q3', 'home_q4', 'home_ot']] = pd.DataFrame(df['home_line_scores'].tolist(), index=df.index)

ValueError: Columns must be same length as key

In [64]:
# First, ensure that the home_line_scores is not NaN or empty and has exactly 4 elements
# df['home_line_scores'] = df['home_line_scores'].apply(lambda x: x if isinstance(x, list) and len(x) == 4 else [None, None, None, None])

# Now, split home_line_scores into separate columns
df[['home_q1', 'home_q2', 'home_q3', 'home_q4']] = pd.DataFrame(df['home_line_scores'].tolist(), index=df.index)


ValueError: Columns must be same length as key

In [58]:
for _, row in df.iterrows():
    if row["home_line_scores"] is [None, None, None, None]:
        print(row["id"])
    else:
        continue


In [44]:
df.columns

Index(['id', 'season', 'week', 'season_type', 'start_date', 'start_time_tbd',
       'completed', 'neutral_site', 'conference_game', 'attendance',
       'venue_id', 'venue', 'home_id', 'home_team', 'home_conference',
       'home_classification', 'home_points', 'home_line_scores',
       'home_postgame_win_probability', 'home_pregame_elo',
       'home_postgame_elo', 'away_id', 'away_team', 'away_conference',
       'away_classification', 'away_points', 'away_line_scores',
       'away_postgame_win_probability', 'away_pregame_elo',
       'away_postgame_elo', 'excitement_index', 'highlights', 'notes',
       'id_venue', 'name', 'city', 'state', 'zip', 'countrycode', 'latitude',
       'longitude', 'capacity', 'dome', 'timezone', 'elevation',
       'constructionyear', 'grass'],
      dtype='object')